# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load cities.csv
cities = pd.read_csv("output_data/cities.csv", encoding="utf-8")
cities.head()


,City,Latitude,Lng,Cloudiness,Country,Date,Humidity,Max Temp,Wind Speed
0,Maltahöhe,-24.83,16.98,0,NaN,1603134554,19,73.06,16.13
1,Mount Isa,-20.73,139.50,0,AU,1603134416,19,77.00,5.82
2,Rikitea,-23.12,-134.97,0,PF,1603134064,79,73.89,14.74
3,Mataura,-46.19,168.86,94,NZ,1603133966,69,50.00,17.00
4,Tuktoyaktuk,69.45,-133.04,92,CA,1603134354,97,22.60,5.59


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps
gmaps.configure(api_key=g_key)

#Variables
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Latitude", "Lng"]]

In [7]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
 Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
    
    

In [25]:
ideal_vacation_df = cities.loc[(cities["Wind Speed"] <= 10) & 
                               (cities["Cloudiness"] < 10) & 
                               (cities['Humidity']< 35) &
                               (cities["Max Temp"] >= 60) & (cities["Max Temp"] <= 95)].dropna()

ideal_vacation_df.head()

,City,Latitude,Lng,Cloudiness,Country,Date,Humidity,Max Temp,Wind Speed
1,Mount Isa,-20.73,139.50,0,AU,1603134416,19,77.00,5.82
178,Kariba,-16.52,28.80,0,ZW,1603134761,25,80.78,5.75
180,Lindsay,36.20,-119.09,1,US,1603134763,34,82.99,3.36
201,Menongue,-14.66,17.69,0,AO,1603134786,18,74.05,4.05
231,Show Low,34.25,-110.03,1,US,1603134818,9,75.20,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [61]:
# Create hotel_df
hotel_df = ideal_vacation_df.loc[:,["City","Country", "Latitude", "Lng"]]

# Add column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df


,City,Country,Latitude,Lng,Hotel Name
1,Mount Isa,AU,-20.73,139.50,
178,Kariba,ZW,-16.52,28.80,
180,Lindsay,US,36.20,-119.09,
201,Menongue,AO,-14.66,17.69,
231,Show Low,US,34.25,-110.03,
260,Kalabo,ZM,-14.97,22.68,
263,Maïné Soroa,NE,13.21,12.02,
281,Maun,BW,-19.98,23.42,
284,Monte Patria,CL,-30.69,-70.95,
290,Hadejia,NG,12.45,10.04,


In [67]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [68]:
for index, row in hotel_raw_df.iterrows():
        
# get city name, lat, lnt from df
    lat = row["Latitude"]
    lng = row["Lng"]
    city_name = row["City"]
    
# add keyword to params dict
    params["location"] = f"{lat},{lng}"

#make API request
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    # Add to Data Frame
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        pass

hotel_df

,City,Country,Latitude,Lng,Hotel Name
1,Mount Isa,AU,-20.73,139.50,Isa Hotel
178,Kariba,ZW,-16.52,28.80,Caribbea Bay Resort
180,Lindsay,US,36.20,-119.09,Super 8 by Wyndham Lindsay Olive Tree
201,Menongue,AO,-14.66,17.69,Ritz Lauca
231,Show Low,US,34.25,-110.03,Hampton Inn & Suites Show Low-Pinetop
260,Kalabo,ZM,-14.97,22.68,Sikakubete Guest House Kalabo Town Council
263,Maïné Soroa,NE,13.21,12.02,NaN
281,Maun,BW,-19.98,23.42,Cresta Maun Hotel
284,Monte Patria,CL,-30.69,-70.95,Hotel Monte Cordillera
290,Hadejia,NG,12.45,10.04,Bababa Guest Inn


In [79]:
# Drop locaitons with no hotel within 5000 meters. 
hotel_df.drop(hotel_df.index[hotel_df['Hotel Name'] == 'NaN'], inplace = True)
hotel_df = hotel_df.reset_index()
hotel_df

,index,City,Country,Latitude,Lng,Hotel Name
0,1,Mount Isa,AU,-20.73,139.50,Isa Hotel
1,178,Kariba,ZW,-16.52,28.80,Caribbea Bay Resort
2,180,Lindsay,US,36.20,-119.09,Super 8 by Wyndham Lindsay Olive Tree
3,201,Menongue,AO,-14.66,17.69,Ritz Lauca
4,231,Show Low,US,34.25,-110.03,Hampton Inn & Suites Show Low-Pinetop
5,260,Kalabo,ZM,-14.97,22.68,Sikakubete Guest House Kalabo Town Council
6,281,Maun,BW,-19.98,23.42,Cresta Maun Hotel
7,284,Monte Patria,CL,-30.69,-70.95,Hotel Monte Cordillera
8,290,Hadejia,NG,12.45,10.04,Bababa Guest Inn
9,354,Saint George,US,37.10,-113.58,Tru By Hilton St. George


In [80]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Lng"]]

In [81]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))